In [92]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')

melb_df['Date'] = pd.to_datetime(melb_df['Date'])

melb_df['Date'].dt.quarter.value_counts()

def is_category(column):
    if column.nunique() < 150:
        return 1
    
columns_to_convert = [col for col in melb_df.columns if col not in ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] and melb_df[col].nunique() < 150]


melb_df[columns_to_convert] = melb_df[columns_to_convert].astype('category')

melb_df.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

In [93]:
sorted_df = melb_df.copy()

m1 = sorted_df['Type'] == 'townhouse'
m2 = sorted_df['Rooms'] > 2



sorted_df = sorted_df[m1 & m2].sort_values(ignore_index=True, by=['Rooms', 'MeanRoomsSquare'], ascending=[True, False])

sorted_df.loc[18,'Price']

1300000.0

In [94]:
m1 = pd.to_datetime('2017-05-01') <= melb_df['Date']
m2 = melb_df['Date'] <= pd.to_datetime('2017-09-01')


melb_df[m1 & m2].groupby('SellerG')['Price'].sum(numeric_only=True).sort_values(ascending=False)

SellerG
Jellis           414121833.0
Nelson           379767500.0
other            365038432.0
Barry            296723050.0
hockingstuart    282834310.0
Marshall         253957888.0
Ray              249572986.0
Buxton           219104000.0
Biggin           110429500.0
Fletchers        108847700.0
Noel              88129800.0
Woodards          85977500.0
Greg              85621000.0
McGrath           78386500.0
Harcourts         68720250.0
Jas               59281600.0
Brad              55955000.0
RT                50498000.0
Kay               48569500.0
Miles             47582000.0
YPA               46354350.0
Hodges            43231000.0
Gary              39138400.0
Sweeney           36882750.0
Stockdale         35409800.0
Raine             30687700.0
RW                29261000.0
Village           26473000.0
HAR               25568000.0
O'Brien           23855508.0
Purplebricks      23401000.0
Love              23365500.0
Williams          23297000.0
Chisholm          23225000.0
Philip

In [95]:
melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    fill_value=0,
    aggfunc='median'
)['unit'].sort_values(ascending=False)

SellerG
Nick             900000
Marshall         715000
Cayzer           707500
Kay              695000
Noel             693250
Buxton           670000
Fletchers        653000
Chisholm         640000
Philip           636000
RT               630000
C21              625500
RW               625500
Moonee           622000
O'Brien          618500
Hodges           605500
Eview            602000
Woodards         600000
Collins          592000
Jellis           591750
Purplebricks     582500
Williams         577000
hockingstuart    566000
Alexkarbon       560000
Miles            555000
Buckingham       553000
Gary             550000
Biggin           548000
Greg             547500
McGrath          542500
other            535000
Nelson           530000
HAR              518000
McDonald         502500
Barry            502000
Harcourts        500000
Ray              497500
Thomson          495000
Douglas          494500
Brad             465000
Rendina          459000
Bells            457500
Jas     

In [96]:
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')

movies = pd.read_csv('data/movies.csv')

dates = pd.read_csv('data/dates.csv')

ratings = pd.concat(objs=[ratings1, ratings2], ignore_index=True)
ratings = ratings.drop_duplicates(ignore_index=True)

ratings = pd.concat(objs=[ratings, dates], axis=1)

merged = ratings.merge(movies, on='movieId', how='left')

merged


,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


In [103]:
data_1 = pd.DataFrame({'Value': [100, 45, 80],
                       'Group': [1, 4, 5]},
                      index = ['I0', 'I1', 'I2']
                     )

data_2 = pd.DataFrame({'Company': ['Google', 'Amazon', 'Facebook'],
                       'Add': ['S0', 'S1', 'S7']},
                      index = ['I0', 'I1', 'I3']
                     )

display(data_1, data_2)

data_1 = data_1.join(data_2, how='left')

data_1

,Value,Group
I0,100,1
I1,45,4
I2,80,5


,Company,Add
I0,Google,S0
I1,Amazon,S1
I3,Facebook,S7


,Value,Group,Company,Add
I0,100,1,Google,S0
I1,45,4,Amazon,S1
I2,80,5,NaN,NaN
